# CHAT_BOT_Using_DeepLearning-Tensorflow

# Importing pacakages

In [1]:
import numpy as np
import nltk
# nltk.download('punkt')
from nltk.stem.porter import PorterStemmer

import random
import json

import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras import layers



# Preprocessing of Data

In [2]:

stemmer = PorterStemmer()

def tokenize(sentence):
    return nltk.word_tokenize(sentence)


def stem(word):
    return stemmer.stem(word.lower())


def bag_of_words(tokenized_sentence, words):
    # stem each word
    sentence_words = [stem(word) for word in tokenized_sentence]
    # initialize bag with 0 for each word
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words: 
            bag[idx] = 1

    return bag


with open('../Dataset.json', 'r') as f:
    dataset = json.load(f)

all_words = []
tags = []
xy = []
# loop through each sentence in our intents patterns
for conv in dataset['conversations']:
    tag = conv['header']
    # add to tag list
    tags.append(tag)
    for pattern in conv['start']:
        # tokenize each word in the sentence
        w = tokenize(pattern)
        # add to our words list
        all_words.extend(w)
        # add to xy pair
        xy.append((w, tag))

# stem and lower each word
ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]
# remove duplicates and sort
all_words = sorted(set(all_words))
tags = sorted(set(tags))

# create training data
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    # X: bag of words for each pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

#convertion of normal data to categorical values
y_train = tf. keras.utils.to_categorical(y_train, 7)



# Model

In [3]:
#model
input_size = len(X_train[0])

def myModel():
    inp = Input(shape=input_size)
    x = Dense(units=8, activation='relu')(inp)
    x = Dense(units=8, activation='relu')(x)
    x = Dense(units=8, activation='relu')(x)
    out = Dense(units=7, activation= tf.keras.activations.softmax)(x)
    
    return Model(inputs = inp, outputs = out, name = 'MY_Model')


model = myModel()
model.summary()

Model: "MY_Model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 54)]              0         
_________________________________________________________________
dense (Dense)                (None, 8)                 440       
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_3 (Dense)              (None, 7)                 63        
Total params: 647
Trainable params: 647
Non-trainable params: 0
_________________________________________________________________


# Prepare the training dataset.

In [4]:
batch_size = 8

# Prepare the training dataset.
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)


# Training Step

In [5]:
epochs = 200
learning_rate = 0.001

# Instantiate an optimizer.
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
# Instantiate a loss function.
loss_fn = tf.keras.losses.CategoricalCrossentropy()

for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))
    
    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch_train, training=True)
            loss_value = loss_fn(y_batch_train, logits)

        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        
        if epoch or epoch==0:
            print (f'Epoch [{epoch+1}/{epochs}], step : {step} Loss: {loss_value:.4f}')

print(f'final loss: {loss_value:.4f}')
        


Start of epoch 0
Epoch [1/200], step : 0 Loss: 1.9890
Epoch [1/200], step : 1 Loss: 2.0030
Epoch [1/200], step : 2 Loss: 1.8943
Epoch [1/200], step : 3 Loss: 1.9312

Start of epoch 1
Epoch [2/200], step : 0 Loss: 2.0283
Epoch [2/200], step : 1 Loss: 1.8361
Epoch [2/200], step : 2 Loss: 1.8955
Epoch [2/200], step : 3 Loss: 2.2972

Start of epoch 2
Epoch [3/200], step : 0 Loss: 1.8848
Epoch [3/200], step : 1 Loss: 1.9484
Epoch [3/200], step : 2 Loss: 1.9802
Epoch [3/200], step : 3 Loss: 1.9599

Start of epoch 3
Epoch [4/200], step : 0 Loss: 1.8804
Epoch [4/200], step : 1 Loss: 1.9167
Epoch [4/200], step : 2 Loss: 2.0128
Epoch [4/200], step : 3 Loss: 1.8738

Start of epoch 4
Epoch [5/200], step : 0 Loss: 1.8818
Epoch [5/200], step : 1 Loss: 1.9948
Epoch [5/200], step : 2 Loss: 1.9136
Epoch [5/200], step : 3 Loss: 1.8662

Start of epoch 5
Epoch [6/200], step : 0 Loss: 1.8830
Epoch [6/200], step : 1 Loss: 1.8552
Epoch [6/200], step : 2 Loss: 2.0110
Epoch [6/200], step : 3 Loss: 1.9297

Sta

Epoch [50/200], step : 3 Loss: 1.7443

Start of epoch 50
Epoch [51/200], step : 0 Loss: 1.2649
Epoch [51/200], step : 1 Loss: 1.5681
Epoch [51/200], step : 2 Loss: 1.4409
Epoch [51/200], step : 3 Loss: 1.4306

Start of epoch 51
Epoch [52/200], step : 0 Loss: 1.4590
Epoch [52/200], step : 1 Loss: 1.4659
Epoch [52/200], step : 2 Loss: 1.3465
Epoch [52/200], step : 3 Loss: 1.1877

Start of epoch 52
Epoch [53/200], step : 0 Loss: 1.2439
Epoch [53/200], step : 1 Loss: 1.4585
Epoch [53/200], step : 2 Loss: 1.4597
Epoch [53/200], step : 3 Loss: 1.3726

Start of epoch 53
Epoch [54/200], step : 0 Loss: 1.2220
Epoch [54/200], step : 1 Loss: 1.3925
Epoch [54/200], step : 2 Loss: 1.4157
Epoch [54/200], step : 3 Loss: 1.6432

Start of epoch 54
Epoch [55/200], step : 0 Loss: 1.5232
Epoch [55/200], step : 1 Loss: 1.1329
Epoch [55/200], step : 2 Loss: 1.4263
Epoch [55/200], step : 3 Loss: 1.1688

Start of epoch 55
Epoch [56/200], step : 0 Loss: 1.4427
Epoch [56/200], step : 1 Loss: 1.1427
Epoch [56/20

Epoch [101/200], step : 1 Loss: 0.3879
Epoch [101/200], step : 2 Loss: 0.4869
Epoch [101/200], step : 3 Loss: 0.5905

Start of epoch 101
Epoch [102/200], step : 0 Loss: 0.4887
Epoch [102/200], step : 1 Loss: 0.5290
Epoch [102/200], step : 2 Loss: 0.3883
Epoch [102/200], step : 3 Loss: 0.9294

Start of epoch 102
Epoch [103/200], step : 0 Loss: 0.4776
Epoch [103/200], step : 1 Loss: 0.4641
Epoch [103/200], step : 2 Loss: 0.5375
Epoch [103/200], step : 3 Loss: 0.4676

Start of epoch 103
Epoch [104/200], step : 0 Loss: 0.5531
Epoch [104/200], step : 1 Loss: 0.4883
Epoch [104/200], step : 2 Loss: 0.4127
Epoch [104/200], step : 3 Loss: 0.4260

Start of epoch 104
Epoch [105/200], step : 0 Loss: 0.5066
Epoch [105/200], step : 1 Loss: 0.4353
Epoch [105/200], step : 2 Loss: 0.4054
Epoch [105/200], step : 3 Loss: 0.6883

Start of epoch 105
Epoch [106/200], step : 0 Loss: 0.5683
Epoch [106/200], step : 1 Loss: 0.3024
Epoch [106/200], step : 2 Loss: 0.4438
Epoch [106/200], step : 3 Loss: 0.6852

St

Epoch [149/200], step : 3 Loss: 0.0998

Start of epoch 149
Epoch [150/200], step : 0 Loss: 0.1616
Epoch [150/200], step : 1 Loss: 0.1826
Epoch [150/200], step : 2 Loss: 0.1284
Epoch [150/200], step : 3 Loss: 0.0935

Start of epoch 150
Epoch [151/200], step : 0 Loss: 0.1532
Epoch [151/200], step : 1 Loss: 0.1455
Epoch [151/200], step : 2 Loss: 0.1664
Epoch [151/200], step : 3 Loss: 0.0765

Start of epoch 151
Epoch [152/200], step : 0 Loss: 0.1650
Epoch [152/200], step : 1 Loss: 0.1444
Epoch [152/200], step : 2 Loss: 0.1492
Epoch [152/200], step : 3 Loss: 0.0564

Start of epoch 152
Epoch [153/200], step : 0 Loss: 0.1318
Epoch [153/200], step : 1 Loss: 0.1681
Epoch [153/200], step : 2 Loss: 0.1336
Epoch [153/200], step : 3 Loss: 0.1136

Start of epoch 153
Epoch [154/200], step : 0 Loss: 0.1650
Epoch [154/200], step : 1 Loss: 0.1561
Epoch [154/200], step : 2 Loss: 0.1127
Epoch [154/200], step : 3 Loss: 0.0716

Start of epoch 154
Epoch [155/200], step : 0 Loss: 0.1185
Epoch [155/200], step 

Epoch [197/200], step : 0 Loss: 0.0462
Epoch [197/200], step : 1 Loss: 0.0455
Epoch [197/200], step : 2 Loss: 0.0729
Epoch [197/200], step : 3 Loss: 0.0569

Start of epoch 197
Epoch [198/200], step : 0 Loss: 0.0471
Epoch [198/200], step : 1 Loss: 0.0439
Epoch [198/200], step : 2 Loss: 0.0697
Epoch [198/200], step : 3 Loss: 0.0597

Start of epoch 198
Epoch [199/200], step : 0 Loss: 0.0626
Epoch [199/200], step : 1 Loss: 0.0532
Epoch [199/200], step : 2 Loss: 0.0378
Epoch [199/200], step : 3 Loss: 0.0768

Start of epoch 199
Epoch [200/200], step : 0 Loss: 0.0594
Epoch [200/200], step : 1 Loss: 0.0574
Epoch [200/200], step : 2 Loss: 0.0439
Epoch [200/200], step : 3 Loss: 0.0348
final loss: 0.0348


# Chat-Bot

In [6]:
with open('../Dataset.json', 'r') as f:
    dataset = json.load(f)

bot_name = "Robot"
print("How may help you! (type 'q' to exit)")
while True:
    sentence = input("You: ")
    if sentence == "q":
        break

    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])

    output = model.predict(X)
    
    tag = tags[np.argmax(output)]
    #print(tag)
    #print(output[0][np.argmax(output)])
    
    if output[0][np.argmax(output)] > 0.75 :
        for conv in dataset['conversations']:
            if tag == conv["header"]:
                print(f"{bot_name}: {random.choice(conv['responses'])}")
    else:
        print(f"{bot_name}: I do not understand...")
        


How may help you! (type 'q' to exit)
You: hi
Robot: Hey :-)
You: tell me joke
Robot: Why did the hipster burn his mouth? He drank the coffee before it was cool.
You: hello
Robot: Hi there, what can I do for you?
You: hiiii
Robot: I do not understand...
You: heloo
Robot: I do not understand...
You: qwertyuiop
Robot: I do not understand...
You: q
